# Laboratorio 10 - Data Science
## Proyecto de Consultoría: Regresión Lineal con Python y Spark

- Nelson García Bravatti
- Christian Echeverría


In [2]:
!pip -q install pyspark

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab DF — Cruceros").getOrCreate()
print("Spark version:", spark.version)

Spark version: 3.5.1


In [8]:
file_path = '/content/barcos_crucero.csv'
df = (
    spark.read
         .option("header", True)
         .option("inferSchema", True)
         .csv(file_path)
)

# Ver las primeras filas del dataframe
df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [10]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [12]:
from pyspark.sql import functions as F, types as T

#Seleccionar columnas numéricas para describir
numeric_types = (T.DoubleType, T.IntegerType, T.LongType, T.FloatType, T.ShortType, T.DecimalType)
num_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, numeric_types)]

#Validar que existan columnas numéricas
if not num_cols:
    raise ValueError("No se detectaron columnas numéricas para describir. Revisa el esquema/lectura del CSV.")

#Descriptivos básicos
desc_basic = df.select(num_cols).describe()
desc_basic.show(truncate=False)

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Age               |Tonnage           |passengers       |length           |cabins            |passenger_density|crew             |
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|count  |158               |158               |158              |158              |158               |158              |158              |
|mean   |15.689873417721518|71.28467088607599 |18.45740506329114|8.130632911392404|8.830000000000005 |39.90094936708861|7.794177215189873|
|stddev |7.615691058751413 |37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615|8.63921711391542 |3.503486564627034|
|min    |4                 |2.329             |0.66             |2.79             |0.33              |17.7             |0.59             |
|max    |48                